#Group 1
Name: Salma Chaaban <br>
Student number: 301216551 <br>
Name: Hodan Ahmed Yusuf<br>
Student number: 301226634<br>
Name: Jonathan Au<br>
Student number: 300827701<br>
Name: Landon Essex<br>
Student number: 301349452<br>
Name: Michael Angelo Cabalinan <br>
Student number: 300924795 <br>
COMP 216 - assignment 11


# Lab 12 – Send and receive messages via MQTT.
You will implement a client to send structured data to a server. You will also implement a client that will receive the structure data.
You will create three separate files for this lab: a subscriber, a publisher and a util. subscriber.


# Wk12a_util.py
This file will have the following:
1.	A start_id that will be initially set to 111 or your favourite number. This will be used to number the payload.
2.	A create_data() function that will create and return a dict. This will be the payload.
3.	A print_data() function that will take a dict and print the parts in a human-readable format.<br>
The following is a sample of the data I used. You should not use mine, you must come up with something of comparable complexity.<br>

```
{
    'id': 112,                                    #sequence number
    'patient': patient,                           #name of patient
    'time': time.asctime(),                       #time this was generated
    'heart_rate': int(random.gauss(80, 1)),       #heart rate
    'respiratory_rate': int(random.gauss(12,2)),  #respiratory rate
    'heart_rate_variability': 65,                 #???
    'body_temperature': random.gauss(99, 0.5),    #temperature
    'blood_pressure': {                           #subkey
        'systolic': int(random.gauss(105,2)),     #systolic pressure
        'diastolic': int(random.gauss(70,1))      #diastolic pressure
    },
    'activity': 'Walking'                         #activity
}
```
We can use a class to model our data, but that would be an overkill. Maybe a python dataclass would be a better fit. However a python dict is able to satisfy all of our needs and it is also light weight.


In [ ]:
import random
import time

start_id = 111

# function that will create and return a dict
def create_data(student):
  global start_id
  data = {
        'id': start_id,                                 # sequence number
        'student': student,                             # name of student
        'time': time.asctime(),                         # time this was generated
        "attendance": random.randint(0, 100),           # attendance percentage
        "study_hours_per_week": random.randint(5, 30),  # study hours per week
        "clubs": ["Academic", "Political", "Athletic"], # participation in clubs
        "test_scores": {
                "MATH210": random.randint(60, 100),     # test scores for different courses
                "COMP216": random.randint(60, 100),
                "GNED500": random.randint(60, 100)
            },
        'program': 'Softwr. Eng. Tech. - AI'
    }
  start_id += 1
  return data

# function that will take a dict and print the parts in a human-readable format
def print_data(data):
    print("ID:", data['id'])
    print("Student:", data['student'])
    print("Time:", data['time'])
    print("Study hours per week:", data['study_hours_per_week'])
    print("Clubs:", ', '.join(data['clubs']))
    print("Test scores:", data['test_scores'])
    print("Program:", data['program'])

#Wk12a_publisher.py
This file will have the following:
1.	Call the above function to obtain a dict.
2.	Convert the above dict to a string. Use json.dumps().
3.	Create a client.
4.	Connect to the server.
5.	Publish to the server.
6.	Print a message.
7.	Close the connection.
It might be advisable to do multiple transmissions (maybe 10) and sleeping at the end of each cycle.


In [ ]:
import json
import time
import paho.mqtt.client as mqtt
from Wk12a_util import create_data

def publish_data():
  client = mqtt.Client()
  client.connect("test.mosquitto.org", 1883, 60)

try:
  for _ in range(10):
    data = create_data()
    data_string = json.dumps(data)
    client.publish("student_data", data_string)
    print("Data Published:", data_string)
    time.sleep(2)

finally:
  client.disconnect()
  print("Disconnected")

#Wk12a_subscriber.py
This file will have the following:
1.	Create a client.
2.	Assign the on_messege delegate to the function in Step 7.
3.	Connect to the server.
4.	Subscribe to the required topic.
5.	Print a message.
6.	Invoke the client loop_forever() method.
7.	Create a function to do the following: (see signature in text or ppt slide)
-	Decode the message.
-	Convert the decoded string to a dict. Use the json.loads() function.
-	Call the function in the first file to print the dictionary.


In [ ]:
import json
import time
import paho.mqtt.client as mqtt
from Wk12a_util import create_data

def connect():
  #Create Client
  client = mqtt.Client()

  #Connect to server
  client.connect("test.mosquitto.org", 1883, 60)

  # On_message callback
  client.on_message = decode

  #Subscribe to topic
  client.subscribe("student_data")

  #Print a message
  print("Subscriber successful.")

  #Invoke the client loop_forever() method
  client.loop_forever()

def decode(client, userdata, message):
  try:
    #Decode
    decoded = message.payload.decode("utf-8")

    #Convert decode
    convert_decode = json.loads(decoded)

    #Call function from 1st file
    call_dict = create_data(convert_decode)

    except Exception as e:
      print("Error while decoding message: ", e)

#Call the connect function
connect()